# DATA 534: Lab 1
## General Lab Instructions

- This assignment is to be completed in python, submitting both a `.ipynb` file (you can add your answers directly to this one) along with a rendered `.md`.
- The demo we saw in class will help you with the basics for this lab. 

# Exercise 1: Creating a prerequisite diagram (5 marks)

Below is a prerequisite chart for the graduate DATA courses:

<img src="MDS-prereq.png" style="width:90%"/>

In this assignment, you will reproduce this graph, or something very similar, by scraping the prerequisite info from https://courses.students.ubc.ca/cs/courseschedule?tname=subj-department&campuscd=UBCO&dept=DATA&pname=subjarea. Note that you might need to append '&campuscd=UBCO' to the end of the URL string for the course pages to display properly 
(eg. https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=101&campuscd=UBCO).

Try loading webpages in a incognito browser window to see the page that the python code will recieve. 

In this assignment, you will implement a simple crawler to crawl and scrape UBC SSC web pages just for grad courses. Grad courses start with 5. 

HINTS: 
    1. Your broser's developer tools is your ally in navigating a web structure.
    2. use `BeautifulSoup` to create a object from the web page
    3. Use `graphviz` to draw the graph


In [2]:
from bs4 import BeautifulSoup
import requests

In [80]:
# TODO: Request the URL (1 mark)
UBCO_url = 'https://courses.students.ubc.ca/cs/courseschedule?tname=subj-department&campuscd=UBCO&dept=DATA&pname=subjarea'
ubco_request = requests.get(UBCO_url)
if ubco_request.status_code == 200:
      print('URL loaded correctly')

# TODO: Create BeautifulSoup object (1 mark)
soup = BeautifulSoup(ubco_request.content, 'html.parser')
# TODO: Get courses (1 mark)
grad_courses = soup.find_all('a')

Grad_courses_url = {}
            
for course in grad_courses:
      if course.text.startswith("DATA 5"):
            # print(course.text)
            Grad_courses_url[course.text] = 'https://courses.students.ubc.ca'+course['href']

URL loaded correctly


In [81]:
Grad_courses_url

# https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=585

{'DATA 505': 'https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=505',
 'DATA 530': 'https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=530',
 'DATA 531': 'https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=531',
 'DATA 532': 'https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=532',
 'DATA 533': 'https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=533',
 'DATA 534': 'https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=534',
 'DATA 540': 'https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=540',
 'DATA 541': 'https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=541',
 'DATA 542': 'https://courses.students.u

In [86]:
print(Grad_courses_url['DATA 550'])
# https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=550

https://courses.students.ubc.ca/cs/courseschedule?pname=subjarea&tname=subj-course&dept=DATA&course=550


In [87]:
page_550 = requests.get(Grad_courses_url['DATA 550'])
soup = BeautifulSoup(page_550.content, 'html.parser')
# course_page

# Data550 = soup.find_all('a')
# Data550
soup


<!DOCTYPE html>

<!--[if IEMobile 7]><html class="iem7 oldie" lang="en"><![endif]-->
<!--[if (IE 7)&!(IEMobile)]><html class="ie7 oldie" lang="en"><![endif]-->
<!--[if (IE 8)&!(IEMobile)]><html class="ie8 oldie" lang="en"><![endif]-->
<!--[if (IE 9)&!(IEMobile)]><html class="ie9" lang="en"><![endif]-->
<!--[[if (gt IE 9)|(gt IEMobile 7)]><!--><html lang="en"><!--<![endif]-->
<!--
 * UBC CLF (Common Look and Feel) v7.0.4
 * Copyright 2012-13 The University of British Columbia
 * UBC Communications and Marketing
 * http://brand.ubc.ca/clf
 */
-->
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta charset="utf-8"/>
<title>DATA 550 - Course - UBC Student Services</title>
<meta content="width=device-width" name="viewport"/>
<meta content="" name="description"/>
<meta content="" name="author"/>
<!-- Needed at top for some UI (responsive timetable buttons) -->
<script src="/static/shared/scripts/jquery-1.8.1.min.js"></script>
<!-- Stylesheets -->
<link href="/s

In [66]:
# course_url = Grad_courses_url['DATA 550']
# print(course_url)
# course_page = BeautifulSoup(Grad_courses_url['DATA 550'])
# course_page = requests.get(course_url)
# soup = BeautifulSoup(course_page.content, 'html.parser')
# pre_reqs = soup.find_all('a')

prerequisites = []

for item in course_page.find_all('p'):
      if 'Pre-reqs:' in item.text:
            for entry in item.find_all('a'):
                  names = names+entry.text + ', '
                  print(names)
                  print('--')
#                   prerequisites.append(pre_req.text)



# prerequisites
# pre_reqs
# course_page

In [35]:

courses = soup.find_all('a')
print(courses)
for pre in courses:
      if pre.text.startswith('Pre'):
            print(pre.text)
      

[<a href="//www.ubc.ca" title="The University of British Columbia (UBC)">The University of British Columbia</a>, <a href="https://cdn.ubc.ca/clf/ref/aplaceofmind" title="UBC a place of mind">UBC - A Place of Mind</a>, <a href="//www.ubc.ca/okanagan/">The University of British Columbia
                        <span class="ubc7-campus" id="ubc7-okanagan-campus">Okanagan campus </span>
</a>, <a href="//www.ubc.ca/" id="ubc7-global-utility-no-script" title="UBC Search">UBC Search</a>, <a class="btn btn-navbar" data-target="#ubc7-unit-navigation" data-toggle="collapse">
<span class="icon-bar"></span>
<span class="icon-bar"></span>
<span class="icon-bar"></span>
</a>, <a href="/cs/courseschedule?pname=welcome&amp;tname=welcome" title="Course Schedule"><span id="ubc7-unit-faculty">Student Service Centre</span><span id="ubc7-unit-identifier">Course Schedule</span></a>, <a class="btn" data-toggle="dropdown">Browse  <div class="ubc7-arrow down-arrow"></div></a>, <a href="/cs/courseschedule?pname

In [20]:
# TODO: Retrieve the page using the link to get pre-reqs (2 mark)
Grad_courses_link = {}
for key in Grad_courses_url:
      url = Grad_courses_url[key]   
      page = requests.get(url)
      courses = soup.find_all('a')
      for course in grad_courses:
            if course.test.startwith('Pre-reqs:')

DATA 505
DATA 530
DATA 531
DATA 532
DATA 533
DATA 534
DATA 540
DATA 541
DATA 542
DATA 543
DATA 550
DATA 551
DATA 552
DATA 553
DATA 570
DATA 571
DATA 572
DATA 573
DATA 580
DATA 581
DATA 582
DATA 583
DATA 585
DATA 586
DATA 589


In [2]:
# TODO: Make graph and export it to pdf (1 mark)


# Exercise 2: Creating a prerequisite diagram using Scrapy (5 marks)

In this exercise, you will implement a simple crawler, using Scrapy, to crawl and scrape UBCO web pages to grab the prerequisite chart for the graduate DATA courses. You should generate the same graph as in Exercise 1.

## Python instructions

You can install Scrapy with `conda install scrapy` or `pip install scrapy`. Here is a link to [scrapy documentation](http://doc.scrapy.org/en/latest/). Additionally, you will need to install `html5lib` and `json` packages.

## Writing your own crawler

This exercise is broken into the following steps:

### Step 1: Creating the crawler folder structure

You will use Scrapy's built in command to generate the crawler folder structure. In the command line (using interfaces such as Terminal in OS X), type `scrapy startproject prereq_project`.

This will generate a directory named prereq_project under your current directory.

### Step 2: Implementing your first scraper

Create a new python file, name it `spider.py` and save it under prereq_project/spiders/ directory. You may fill in the file with the code below:

```
import scrapy
import html5lib
from bs4 import BeautifulSoup

class MySpider(scrapy.Spider):

    name = "mySpider"
    allowed_domains = ["ubc.ca"]
    start_urls = [
        "https://courses.students.ubc.ca/cs/courseschedule?tname=subj-department&campuscd=UBCO&dept=DATA&pname=subjarea"
    ]
    custom_settings = {
        'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36',
    }

    def __init__(self):
        self.log_file = open('log_file', 'w')

    def parse(self, response):
      soup = BeautifulSoup(response.text, 'html5lib')
      return

    def parse_course_details(self, response):
        pass
```
The spider requires specification of the name, domain that it is allowed to crawl, and a list of URLs from which the spider is to begin crawling. You can run the crawler with the following command (although it will do nothing):

`scrapy crawl mySpider`

The scrapy crawler works by sending an HTTP request for each URL in the start_urls field. Once it receives a response, the default callback function, parse(response), is invoked with a [Response](http://doc.scrapy.org/en/1.6/topics/request-response.html#scrapy.http.Response) object passed into the function.

Your task in this step is to complete the implementation of the parse(response) function so that you can retrieve the list of all data science courses (DATA 5XX) as well as the links to the course details. The first step of creating a BeautifulSoup object, soup to parse the response has been completed for you.

We recommend you to use `self.log_file` to output the list of courses that you retrieved -- this will be helpful as you debug your code.

### Step 3: Structuring the scraped data

Scrapy provides a mechanism for structuring the scraped data using [Items](http://doc.scrapy.org/en/1.6/topics/items.html).

You will define your own Item class by subclassing Scrapy's Item class. Open prereq_project/items.py. The file contains a simple definition of TestItem class. You may rename the class to CourseItem.

Item class comes with a default [Field](http://doc.scrapy.org/en/1.6/topics/items.html#scrapy.item.Field) object, which is essentially a Python dictionary. We want to declare at least two fields: `course_name` and `url_link`.

Now, modify the `spider.py`, in particular, parse(response) function to use `CourseItem class` and store the course name and the links in the fields that you just created. Once you are done, run the crawler via the command,

`scrapy crawl mySpider -o courses.json`

Check the `courses.json` file to ensure that your implementation is correct.

### Step 4: Crawling the link

To retrieve the prerequisite for each of the courses, you will need to follow the links associated with each of the courses. This requires you to make an HTTP [Request](http://doc.scrapy.org/en/1.6/topics/request-response.html#request-objects).

Your task here is to make an HTTP request on each of the links for each of the courses that you retrieved from Steps 2 and 3, and fill out the second callback function, `parse_course_details(response)`, to retrieve the prerequisities for each of the courses.

Output the retrieved course pre-requisite to the `courses.json` file. Note that you will have to change `items.py` file as you will need to store the list of prerequisite courses.

### Step 5. Generate the graphs

Use the [Python wrapper](https://pypi.python.org/pypi/graphviz) around [graphviz](http://www.graphviz.org/) to turn this into a picture. You should have graphviz but you may need to install the Python wrapper, which can be achieved with conda install graphviz.

### Step 6: Submit the code in canvas

Zip the entire project and submit the code. Make sure that we can run your code by running from our command line:

`scrapy crawl mySpider -o courses.json`

# Exercise 3 (5 marks)

All the Game of Thrones episodes are listed, by season, in the following URL: https://en.wikipedia.org/wiki/List_of_Game_of_Thrones_episodes

Unfortunately, the running time of each episode is not available. However, in the link provided for each episodes (e.g., https://en.wikipedia.org/wiki/Dragonstone_(Game_of_Thrones)) there is the running time of the respective episode. Collect the episodes' titles, season, number of U.S. viewers, and running time from wikipedia and create a pandas dataframe with the information collected.

In [ ]:
# TODO